<a href="https://colab.research.google.com/github/ratmcu/wiki_ner/blob/master/reusable_annotator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install pyahocorasick
!pip install fuzzyset
from ahocorasick import Automaton
import fuzzyset
import urllib
from bs4 import BeautifulSoup
from nltk.tokenize import sent_tokenize
import nltk
import re
import os
import pandas as pd
nltk.download('punkt')

     |████████████████████████████████| 317kB 10.2MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.0-cp36-cp36m-linux_x86_64.whl size=81707 sha256=a4352674a98f7829495781652cec2ce7e1ebc61c9a8f6e282adef58e11058bb5
  Stored in directory: /root/.cache/pip/wheels/0a/90/61/87a55f5b459792fbb2b7ba6b31721b06ff5cf6bde541b40994
Successfully built pyahocorasick
     |████████████████████████████████| 378kB 8.7MB/s 
     |████████████████████████████████| 51kB 25.0MB/s 
  Created wheel for fuzzyset: filename=fuzzyset-0.0.19-cp36-cp36m-linux_x86_64.whl size=167538 sha256=8389014e60475bca8d4c561016f1e05b320ce1c75a1819f50891f3cdb35c86f5
  Stored in directory: /root/.cache/pip/wheels/d8/36/9a/8f1cac047c7c3b03dce3d5434ed0088bfd8da8aeca615dfb4c
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144661 sha256=f8ec3484090d969b23c66e554adffde3abb62fdcf33981ae3b283b73ec565319
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/66

True

In [0]:
class PageContents():
  def __init__(self, url):
      self.quote_page = url
      try:
        self.page = urllib.request.urlopen(self.quote_page,  timeout = 5)
        self.soup = BeautifulSoup(self.page, 'html.parser')
      except:
        self.table = False
      
#       side_pane = self.soup.find('tbody')#.find_all('tr')
      try:
        self.side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'}) # or infobox biography vcard
        if not self.side_pane:
            self.side_pane = self.soup.find('table', attrs={'class': 'infobox biography vcard'})
        self.table_entry_list = self.side_pane.find('tbody').find_all('tr')
        self.table = True
      except:
        self.table = False
#       print(self.table_entry_list)
#       self.table_entry_list = self.soup.find('tbody').find_all('tr')
   
  def get_party(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
      party = ''
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
              if child.text == 'Political party':
                  party = child.next_sibling.contents[0].get('title')
#                   print(party)
             
  def get_name(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
      names = []
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
#               print(child)
              try:
                if child.text == 'Born':
                    div_list = child.next_sibling.find_all('div')
                    for i, item in enumerate(div_list):
#                       try:
                      t = item['class']
#                       print(t)
                      if t[0] == 'nickname':
#                         print(item.text)
                        names.append(item.text)
#                       except:
#                         pass
                    break
              except:
                pass  
      if len(names) != 0:
        return names
      else:
#         print('no name found')
#         print(self.soup.find('div', attrs={'class': 'fn'}).text)
        try:
          names.append(self.soup.find('div', attrs={'class': 'fn'}).text)
#         div_list = self.soup.find('tbody').find_all('div')
#         for i, item in enumerate(div_list):
#           try:
#             t = item['class']
# #             print(t)
#             if t[0] == 'fn':
# #               print(item.contents[0])
#               names.append(item.contents[0])
        except:
          pass
      return names
  def get_birth_date(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
      date = []
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
            try:
              if child.text == 'Born':
                  for i, item in enumerate(child.next_sibling.contents):
#                     try:
                    t = item.span['class']
#                       print(t)
                    if t[0] == 'bday':
#                         print(child.next_sibling.contents[i+1])
                      date.append(child.next_sibling.contents[i+1])
#                     except:
#                       pass
                  break
            except:
              pass
      return date  
    
  def get_birth_place(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
      party = []
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
              try:
                  if child.text == 'Born':
    #                   print(child.next_sibling.contents)
    #                 sibling = find all 
    #                 for content in child.next_sibling.contents:
    #                   if 
    #                   try:
                      party.append(child.next_sibling.contents[-1].text)
                      break
    #                   except:
    #                     pass
              except:
                  pass
      return party
    
  def get_children(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
      party = []
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
#               print(child.text)
              try:
                  if child.text == 'Children':
                      children = child.next_sibling.find_all('a')
    #                   print([child for child in children])
    #                   print(child.next_sibling.contents[0].text)
    #                   print(child.next_sibling.contents)
    #                   sen = re.sub('[[0-9]*]', '', sentence) #remove the reference brackets
                      party.extend([re.sub('[[0-9]*]', '', child.text) for child in children])
              except:
                  pass
      if party != ['']:
#         print(party)
        return party
      else:
        return self.get_children_alternate()
      
  def get_children_alternate(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
      party = []
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
#               print(child.text)
              if child.text == 'Children':
#                   children = child.next_sibling.find_all('a')   
#                   print('-----')
#                   print(child.next_sibling.contents)
                  for kid in child.next_sibling.contents:
#                     print(type(kid))
#                     print('---+++_++_+++')
                    try:
                      a = BeautifulSoup(kid, "html.parser").find_all('a')
#                       print('printing a s')
#                       print(a)
                      if not bool(a):
#                         print(kid)
                        party.append(kid)
#                       else:
#                         print('html kid!')
#                         party.append(re.sub('[[0-9]*]', '', kid.text))
                    except:
#                       print(kid.text)
#                       party.append(re.sub('[[0-9]*]', '', kid.text))
                        pass
      return party
    
  def get_spouses(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_self.all('tr')
      party = []
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
              try:
                text = child.text
              except:
                continue
              if text == 'Spouse(s)':
                try:
#                   children = child.next_sibling.contents[0].find_all('a')
                  children = child.next_sibling.find_all('a')
#                   print([child.text for child in children])
                  if len(children) == 0:
#                     print(child.next_sibling.text)
                    party.extend([child.next_sibling.text])
                  else:
                    party.extend([child.text for child in children])
                except:
#                   print(child.next_sibling.contents)
                  return child.next_sibling.contents
                  pass
                break
      return party
  
  def get_parents(self):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       side_pane = self.soup.find('tbody')#.find_all('tr')
#       table_entry_list = side_pane.find_all('tr')
      party = []
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
              try:
                text = child.text
              except:
                continue
              if text == 'Parents':
                  children = child.next_sibling.find_all('a')
#                   print([child.text for child in children])
                  party.extend([child.text for child in children])
      return party
  
  def get_any(self, field):
#       side_pane = self.soup.find('table', attrs={'class': 'infobox vcard'})
#       table_entry_list = side_pane.find_all('tr')
      party = []
      for table_entry in self.table_entry_list:
          for child in table_entry.children:
              if child.text == field:
                  party.append(child.next_sibling.contents[0].text)
      return party

  def get_text(self):
    '''TODO: filter out the table entries and stuff that are coming under paragraph tags as text'''

#     nltk.download('punkt')
    if not self.table:
      return []
    paragraphs = self.soup.find_all('p')
    paragraphs_list = []
    for par in paragraphs:
      paragraphs_list.append(sent_tokenize(par.text))
    #   len(sent_tokenize_list)
    # paragraphs_list

    text = []
    for paragraph in paragraphs_list: 
      for sentence in paragraph:
        sen = re.sub('[[0-9]*]', '', sentence) #remove the reference brackets
        sen = re.sub('\xa0', ' ', sen) #remove the reference brackets
        text.append(sen)
    #     print(sen)  
    return text

  def get_text_chunk(self):
    '''TODO: filter out the table entries that are coming under paragraph tags'''
#     nltk.download('punkt')
    if not self.table:
      return []
    paragraphs = self.soup.find_all('p')
    paragraphs_list = []
    for par in paragraphs:
      paragraphs_list.append(sent_tokenize(par.text))
    #   len(sent_tokenize_list)
    # paragraphs_list

    text = ''
    for paragraph in paragraphs_list: 
      for sentence in paragraph:
        sen = re.sub('[[0-9]*]', '', sentence) #remove the reference brackets
        text = text + ' ' + sen
    #     print(sen)  
    return text

  def get_info_dict(self):
    info_dict = {}
    if not self.table:
      info_dict['NAME'] = ['']
      info_dict['BIRTH_DATE'] = ['']
      info_dict['BIRTH_PLACE'] = ['']
      info_dict['CHILDREN'] = ['']
      info_dict['SPOUSES'] = ['']
      info_dict['PARENTS'] = ['']
      return info_dict
    
    info_dict['NAME'] = self.get_name()
    info_dict['BIRTH_DATE'] = self.get_birth_date()
    info_dict['BIRTH_PLACE'] = self.get_birth_place()
    info_dict['CHILDREN'] = self.get_children()
    info_dict['SPOUSES'] = self.get_spouses()
    info_dict['PARENTS'] = self.get_parents()
    return info_dict
  
  def get_contents(self):
    return self.get_info_dict(), self.get_text()
  
def aggregate_parts_and_annotate(annotated_list, txt, words_within = 2):
  annot_list = []
#   print(len(annotated_list))
  if len(annotated_list) > 1:
    matched = set([ annotated_list[i][3][1][1] for i in range(1,len(annotated_list)-1)])
    sub_sentence = txt[annotated_list[0][0]][annotated_list[0][3][0]+1:annotated_list[-1][3][0]-len(annotated_list[-1][3][1][1])]
    enclosed = set(sub_sentence.split()) # could there be an elegant way to do this?? 
    difference = enclosed.difference(matched)
    if len(difference)>words_within:
#       pass
      for annot in annotated_list:
        annot_list.append((annotated_list[0][0],(annot[3][0]-len(annot[3][1][1])+1, annot[3][0]+1), annot[4])) # [sentence number, (start char, end char), annotation type]
    else:
        annot_list.append((annotated_list[0][0],(annotated_list[0][3][0]-len(annotated_list[0][3][1][1])+1,annotated_list[-1][3][0]+1),annotated_list[0][4]))
#     print(enclosed.difference(matched)) 
#     print(sub_sentence.split())
  elif len(annotated_list) == 1:
    annot_list.append((annotated_list[0][0],(annotated_list[0][3][0]-len(annotated_list[0][3][1][1])+1, annotated_list[0][3][0]+1), annotated_list[0][4])) # [sentnce number, (start char, end char), anootation type]
  else:
    return []
    #   print(annot_list)
  return annot_list

def annotate_text(info_dict, txt):
  annotations = []
  keys = list(info_dict.keys())
  for key in keys: # for every entry in the info dictionary 
    annotate_list = []
    for i, sentence in enumerate(txt): # get a sentence from the text
        fz = fuzzyset.FuzzySet(use_levenshtein=False)
        for word in sentence.split(): # add all the words into a fuzzy set from that sentence
            fz.add(word)        
    #     A=Automaton()
        for j, detail in enumerate(info_dict[key]): # get a detail in the list under an info line
          annotate_sub = []
          matched_list = []
          A = Automaton()
          tokens_in_detail = len(detail.split()) # split the detail into words
          for word in detail.split(): # get a word from the detail
    #         print(word)
            result = fz.get(word)    # get the matching 
            if(result and result[0][0]>=0.5 and not len(result[0][1])/2 < len(word)/2): #if the matching confidence is high and word length is high
              matched_list.append((word, result))
  #             print(result[0][1], word)
  #         if len(matched_list) == tokens_in_detail:
          if abs(len(matched_list) - tokens_in_detail) < 1 and len(matched_list)!=0:
            for matches in matched_list:
                for match in matches[1]:
                    A.add_word(match[1], (matches[0], match[1]))
  #                   print(match[1], '--->', match)
            A.make_automaton()
    #         print(sentence)
            for item in A.iter(sentence):
                # print(item, item[0]+1- len(item[1][1]), item[0] , sentence)
                # print(item, item[0]+1- len(item[1][1]), item[0])
                annotate_sub.append([i, j, detail, item, key])
            if len(annotate_sub) != 0:
              annotate_list.append(annotate_sub)    
#     annotation_minimized = []
    for list_sub in annotate_list:
      if list_sub[0][2].split()[0] != list_sub[0][3][1][0]:
        list_sub.pop(0)
#       annotation_minimized.extend(aggregate_parts_and_annotate(list_sub,txt))
      annotations.extend(aggregate_parts_and_annotate(list_sub, txt))

#   for list_sub in annotations: #uncomment to print the annotation list!
#     print(list_sub, txt[list_sub[0]], '\'', txt[list_sub[0]][list_sub[1][0]:list_sub[1][1]], '\'')
  return annotations

def takeSentenceNumber(annot):
  return annot[0]

# testing sentence scraping

In [0]:
#experiment_code

# page_cnts = PageContents('https://en.wikipedia.org/wiki/Hafizullah_Amin')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Rexhep_Meidani')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Mohamed_Boudiaf')
page_cnts = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Barack_Obama')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Itamar_Franco')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Rivo_Rakotovao')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Robert_Mugabe')
# page_cnts.get_text()
for sent in page_cnts.get_text():
    print(sent)

Donald John Trump (born June 14, 1946) is the 45th and current president of the United States.
Before entering politics, he was a businessman and television personality.
Trump was born and raised in the New York City borough of Queens, and received a B.S.
degree in economics from the Wharton School at the University of Pennsylvania.
He took charge of his family's real-estate business in 1971, renamed it The Trump Organization, and expanded it from Queens and Brooklyn into Manhattan.
The company built or renovated skyscrapers, hotels, casinos, and golf courses.
Trump later started various side ventures, mostly by licensing his name.
He co-authored several books, including The Art of the Deal.
He owned the Miss Universe and Miss USA beauty pageants from 1996 to 2015, and produced and hosted The Apprentice, a reality television show, from 2003 to 2015.
Forbes estimates his net worth to be $3.1 billion.
Trump entered the 2016 presidential race as a Republican and defeated 16 other candidat

In [0]:
# import spacy
# from spacy import displacy
# nlp = spacy.load("en_core_web_sm")
# doc = nlp(page_cnts.get_text_chunk())
# sents = [sentence for sentence in doc.sents]
# sents

[ Donald John Trump (born June 14, 1946) is the 45th and current president of the United States.,
 Before entering politics, he was a businessman and television personality.,
 Trump was born and raised in the New York City borough of Queens, and received a B.S. degree in economics from the Wharton School at the University of Pennsylvania.,
 He took charge of his family's real-estate business in 1971, renamed it The Trump Organization, and expanded it from Queens and Brooklyn into Manhattan.,
 The company built or renovated skyscrapers, hotels, casinos, and golf courses.,
 Trump later started various side ventures, mostly by licensing his name.,
 Trump is credited as co-author of  several books, including The Art of the Deal.,
 He owned the Miss Universe and Miss USA beauty pageants from 1996 to 2015, and produced and hosted The Apprentice, a reality television show, from 2003 to 2015.,
 Forbes estimates his net worth to be $3.1 billion.,
 Trump entered the 2016 presidential race as a R

In [0]:
#experiment_code

# page_cnts = PageContents('https://en.wikipedia.org/wiki/Hafizullah_Amin')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Rexhep_Meidani')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Mohamed_Boudiaf')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Donald_Trump')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Barack_Obama')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Itamar_Franco')
# page_cnts = PageContents('https://en.wikipedia.org/wiki/Rivo_Rakotovao')
page_cnts = PageContents('https://en.wikipedia.org/wiki/Robert_Mugabe')

# annotate_text(page_cnts.get_contents())
a,b = page_cnts.get_contents()
c = annotate_text(a,b)
a

c.sort(key=takeSentenceNumber)

# null_annot = (None, (None, None), 'NONE')
# annot_list = []

# for i, sentence in enumerate(b):
#   sentence_has_annotations = False
#   for j in range(len(c)):
#     if  c[j][0] > i:  # break if the annotation list goes beyond the current sentence
#       break;
#     if c[j][0] == i:
#       annot_list.append(c[j])
#       sentence_has_annotations = True
# #       print(sentence,i,c[annotation_idx])
#   if not sentence_has_annotations:
#     annot_list.append(null_annot)

sentences = {'sentence': b}
annotations = {'annotation': c}
df = pd.DataFrame(data = sentences)
df.to_csv (r'sentences.csv', index = None, header=True)
df = pd.DataFrame(data = annotations)
df.to_csv (r'annotations.csv', index = None, header=True)

In [0]:
#experiment_code
import pickle
import time
class hashabledict(dict):
    def __hash__(self):
        return hash(tuple(sorted(self.items())))
      
if not os.path.exists('dataset'):
    os.makedirs('dataset')
    
category = 'politicians'

if not os.path.exists('politicians'):
    os.makedirs('politicians')
    
with open('president_list.pkl', 'rb') as f:
    prsdnt_list = pickle.load(f)

In [0]:
#experiment_code
#generation_code
#this block of code created the total dataset and saves it to the disk and archives it as dataset.tar.gz
# prsdnt_list = ['https://en.wikipedia.org/wiki/Mohamed_Boudiaf']
for cntry in prsdnt_list:
#   time.sleep(1)
#   if not os.path.exists('politicians' + '/' + cntry['country']):
#     os.makedirs('politicians' + '/' + cntry['country'])
    for prsdnt in cntry['presidents']:
        if cntry['country'] and prsdnt['name']:
            path = 'dataset' + '/' + category + '/' + cntry['country'] + '/' +  prsdnt['name']
    else:
        continue
    if not os.path.exists(path):
        os.makedirs(path)
    else:
        continue
    print(prsdnt['url'])
    page_cnts = PageContents(prsdnt['url'])
#     print(prsdnt['url'])
    a,b = page_cnts.get_contents()
    print(a)
    c = annotate_text(a,b)
    c.sort(key=takeSentenceNumber)
    sentences = {'sentence': b}
    annotations = {'annotation': c}
    df = pd.DataFrame(data = sentences)
    df.to_csv (r'{0}/sentences.csv'.format(path), index = None, header=True)
    df = pd.DataFrame(data = annotations)
    df.to_csv (r'{0}/annotations.csv'.format(path), index = None, header=True)
#     df = pd.DataFrame(data = prsdnt)
#     df.to_csv (r'{0}/president.csv'.format(path), index = None, header=True)
    with open('{0}/meta_data.pkl'.format(path), 'wb') as f:
        pickle.dump(prsdnt, f)
#     annotation_idx = 0
#     null_annot = (None, (None, None), 'NONE')
#     annot_list = []
#     try:
#     print(c)
#     for i, sentence in enumerate(b):
#       if annotation_idx < len(c) and c[annotation_idx][0] == i:
#         annot_list.append(c[annotation_idx])
#         annotation_idx = annotation_idx + 1
#       else:
#         annot_list.append(null_annot)
# #     except:
# #       print(page_cnts.soup)
#     d = {'sentence': b, 'annotation': annot_list}
#     df = pd.DataFrame(data=d)
#     df.to_csv (r'{0}/annotation_dataframe.csv'.format(path), index = None, header=True)

import tarfile

tar =  tarfile.open('dataset.tar.gz',mode='w:gz')
tar.add('dataset')
tar.close()

https://en.wikipedia.org/wiki/Joachim_Gauck
{'NAME': ['Joachim Gauck'], 'BIRTH_DATE': ['24 January 1940'], 'BIRTH_PLACE': ['Germany'], 'CHILDREN': [], 'SPOUSES': ['Gerhild Radtke  (1959–1991, separated)'], 'PARENTS': []}
https://en.wikipedia.org/wiki/Roman_Herzog
{'NAME': ['Roman Herzog'], 'BIRTH_DATE': ['5 April 1934'], 'BIRTH_PLACE': ['Germany'], 'CHILDREN': [], 'SPOUSES': ['Christiane Krauß'], 'PARENTS': []}
https://en.wikipedia.org/wiki/Nana_Akufo-Addo
{'NAME': ['William Addo Dankwa Akufo-Addo'], 'BIRTH_DATE': [], 'BIRTH_PLACE': ['(now Ghana)'], 'CHILDREN': [], 'SPOUSES': ['Rebecca Griffiths-Randolph'], 'PARENTS': []}
https://en.wikipedia.org/wiki/John_Agyekum_Kufuor
{'NAME': ['John Kufuor'], 'BIRTH_DATE': ['8 December 1938'], 'BIRTH_PLACE': ['Colony of the Gold Coast'], 'CHILDREN': [], 'SPOUSES': ['Theresa Kufuor'], 'PARENTS': []}
https://en.wikipedia.org/wiki/John_Mahama
{'NAME': ['His Excellency John Dramani Mahama'], 'BIRTH_DATE': ['29 November 1958'], 'BIRTH_PLACE': ['Ghana'],

In [0]:
#experiment_code
import pandas as pd

annotation_idx = 0
null_annot = (None, (None, None), 'NONE')
annot_list = []
for i, sentence in enumerate(b):
  if c[annotation_idx][0] == i:
    annot_list.append(c[annotation_idx])
    annotation_idx = annotation_idx + 1
  else:
    annot_list.append(null_annot)
    
d = {'sentence': b, 'annotation': annot_list}
df = pd.DataFrame(data=d)
# df

df.to_csv (r'annotation_dataframe.csv', index = None, header=True)

In [0]:
#experiment_code
quote_page = 'https://en.wikipedia.org/wiki/Mike_Tyson'

page = urllib.request.urlopen(quote_page)
soup = BeautifulSoup(page, 'html.parser')

side_pane = soup.find('table', attrs={'class': 'infobox biography vcard'}) # or infobox biography vcard
if side_pane:
    table_entry_list = side_pane.find('tbody').find_all('tr')
else:
    print('no infobox')
